In [1]:
# Dependencies and Setup
import pandas as pd

In [2]:
# load csv to DF_main
df_main = pd.read_csv('Resources/purchase_data.csv')

In [3]:
df_main.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [4]:
# Player Count
player_count = [len(df_main['SN'].unique())]
df = pd.DataFrame.from_dict({'Total Players':player_count})
df

,Total Players
0,576


In [5]:
'''
Purchasing Analysis (Total)

'''

# Number of Unique Items
num_uniq_items = round(len(df_main['Item ID'].unique()),0)
df_main['Number of Unique Items'] = num_uniq_items

# Average Price
avg_price = round(df_main['Price'].mean(),2)

# Number of Purchases
num_purchases = df_main['Purchase ID'].count()

# Total Revenue
tot_rev = df_main['Price'].sum()

# Build DataFrame, add values
df = pd.DataFrame(columns=['Number of Unique Items','Average Price','Number of Purchases',
                                             'Total Revenue'])
df.loc[0] = [num_uniq_items,avg_price,num_purchases,tot_rev]

# Match Formatting
df['Number of Unique Items'] = df['Number of Unique Items'].apply(lambda x: int(x))
df['Number of Purchases'] = df['Number of Purchases'].apply(lambda x: int(x))
df['Average Price'] = df['Average Price'].map('${:.2f}'.format)
df['Total Revenue'] = df['Total Revenue'].map('${:,.2f}'.format)

df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [6]:
'''
Gender Demographics

'''
# Fresh DF
df = df_main.drop_duplicates('SN', keep='last')
df = df.groupby('Gender')
df = pd.DataFrame(df['Purchase ID'].count())

# Build and clean up columns
df.rename(columns={'Purchase ID':'Total Count'},inplace=True)
df.sort_values('Total Count',ascending=False,inplace=True)
total_sn = df['Total Count'].sum()
df['Percentage of Players'] = df.apply(lambda x: (x/total_sn)*100)
df['Percentage of Players'] = df['Percentage of Players'].map('{:.2f}%'.format)

# Save unique Gender SN totals
unique_male_num = df[df.index=='Male']['Total Count'][0]
unique_female_num = df[df.index=='Female']['Total Count'][0]
unique_other_num = df[df.index=='Other / Non-Disclosed']['Total Count'][0]


df

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [7]:
'''
Purchasing Analysis (Gender)
'''
# Build DF grouped by Gender

df = df_main.groupby('Gender')

# Purchase Count
purchase_count = df['Purchase ID'].count()

# Average Purchase Price
avg_price = df['Price'].mean()

# Total Purchase Value
total_purch_val = df['Price'].sum()


# Avg Total Purchase per Person
unique_gender_count = [unique_female_num,unique_male_num,unique_other_num]

avg_total_purch_by_person = []
x = 0
for item in total_purch_val:
    avg_item = item/unique_gender_count[x]
    avg_total_purch_by_person.append(round(avg_item,2))
    x += 1

# Build Data Frame

data_zip = zip(purchase_count,avg_price,total_purch_val)
new_df = pd.DataFrame(data_zip,columns=['Purchase Count','Average Purchase Price','Total Purchase Value'])

new_df.index = ['Female','Male','Other / Non-Disclosed']
new_df['Avg Total Purchase per Person'] = avg_total_purch_by_person

# Match Formatting
new_df['Average Purchase Price'] = new_df['Average Purchase Price'].map('${:.2f}'.format)
new_df['Total Purchase Value'] = new_df['Total Purchase Value'].map('${:.2f}'.format)
new_df['Avg Total Purchase per Person'] = new_df['Avg Total Purchase per Person'].map('${:.2f}'.format)

new_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [8]:
'''
Age Demographics
'''
# Fresh DF drop duplicate SN's
df = df_main.drop_duplicates('SN', keep='last')

# Create age bins and buckets

bins = [0,9,14,19,24,29,34,39,999]
group_names = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']

# create age categorical column
df['Group Names'] = pd.cut(df['Age'], bins, labels=group_names,include_lowest=True)

# Create dataframe, create percentage column, rename column
new_df = pd.DataFrame(df.groupby('Group Names')['Purchase ID'].count())
new_df['Percentage of Players'] = new_df.apply(lambda x: (x/total_sn)*100)
new_df.rename(columns={'Purchase ID':'Total Count'},inplace=True)

# save for Age Analysis
age_total_counts = new_df['Total Count']


# format columns
new_df['Percentage of Players'] = new_df['Percentage of Players'].map('{:.2f}%'.format)

new_df

C:\Users\rbvan\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,Total Count,Percentage of Players
Group Names,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [9]:
'''
Purchasing Analysis (Age)

'''
# fresh df
df = df_main

# Borrow bins and labels to create age categorical column
df['Group Names'] = pd.cut(df['Age'], bins, labels=group_names,include_lowest=True)

# Purchase Count
purchase_count = df.groupby('Group Names')['Purchase ID'].count()

# Average Purchase Price
avg_price = df.groupby('Group Names')['Price'].mean()

# Total Purchase Value
total_purch_val = df.groupby('Group Names')['Price'].sum()

# Build DataFrame
data_zip = zip(purchase_count,avg_price,total_purch_val)
new_df = pd.DataFrame(data_zip,columns=['Purchase Count','Average Purchase Price','Total Purchase Value'])

# create list of average purchase by age
avg_total_purch_by_age = []
x = 0
for item in total_purch_val:
    avg_item = item/age_total_counts[x]
    avg_total_purch_by_age.append(round(avg_item,2))
    x += 1

# Set Index and add Avg Total column
new_df.index = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']

new_df['Avg Total Purchase per Person'] = avg_total_purch_by_age

# format columns
new_df['Average Purchase Price'] = new_df['Average Purchase Price'].map('${:.2f}'.format)
new_df['Total Purchase Value'] = new_df['Total Purchase Value'].map('${:.2f}'.format)
new_df['Avg Total Purchase per Person'] = new_df['Avg Total Purchase per Person'].map('${:.2f}'.format)

new_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [10]:
'''
Top Spenders
'''
# Groupby Screen Name
df = df_main.groupby('SN')

# Total Purchase Value
top_five_df = df['Price'].sum().sort_values(ascending=False)[0:5]

# Total Purchase Count
tot_purch_count = df['Purchase ID'].count()

# Merge DF
new_df = pd.merge(top_five_df,tot_purch_count,left_index=True,right_index=True)

# Average Purchase Price
new_df['Average Purchase Price'] = new_df['Price']/new_df['Purchase ID']

# rename and order columns
new_df.rename(columns = {'Purchase ID':'Purchase Count','Price':'Total Purchase Value'},inplace=True)
new_df = new_df[['Purchase Count','Average Purchase Price','Total Purchase Value']]

# format columns
new_df['Average Purchase Price'] = new_df['Average Purchase Price'].map('${:.2f}'.format)
new_df['Total Purchase Value'] = new_df['Total Purchase Value'].map('${:.2f}'.format)

new_df

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [11]:
'''
Most Popular Items
'''

df = df_main.groupby(['Item ID','Item Name'])

# Total Purchase Value
tot_purch_value = df['Price'].sum()

# Total Purchase Count
tot_purch_count = df['Purchase ID'].count()

# Average Item Price
avg_item_price = df['Price'].mean()

# Merge DFs
new_df = pd.merge(tot_purch_value,tot_purch_count,left_index=True,right_index=True)
new_df = pd.merge(new_df,avg_item_price,left_index=True,right_index=True)

# save df for most profitable Items
df_template = new_df


# sort DF , rename and order columns
new_df = new_df.sort_values('Purchase ID',ascending=False)
new_df.rename(columns={'Price_x':'Total Purchase Value','Purchase ID':'Purchase Count','Price_y':'Item Price'},inplace=True)
new_df = new_df[['Purchase Count','Item Price','Total Purchase Value']]

# format columns
new_df['Item Price'] = new_df['Item Price'].map('${:.2f}'.format)
new_df['Total Purchase Value'] = new_df['Total Purchase Value'].map('${:.2f}'.format)

# preview the data
new_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [12]:
'''
Most Profitable Items
'''

# borrow original most popular items df template
new_df = df_template

# sort DF , rename and order columns
new_df = new_df.sort_values('Price_x',ascending=False)
new_df.rename(columns={'Price_x':'Total Purchase Value','Purchase ID':'Purchase Count','Price_y':'Item Price'},inplace=True)
new_df = new_df[['Purchase Count','Item Price','Total Purchase Value']]

# format columns
new_df['Item Price'] = new_df['Item Price'].map('${:.2f}'.format)
new_df['Total Purchase Value'] = new_df['Total Purchase Value'].map('${:.2f}'.format)

# preview the data
new_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
